In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sentence_transformers import SentenceTransformer
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv("../../data/final_data.csv")
df.head()

,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,...,company_industry,job_description,salary,job_description_cleaned,annotations,skills,career_level_norm,experience_years,experience_bucket,education_level_norm
0,10,Tax Supervisor,Banten,IDR,Supervisor/Koordinator,3 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Akuntansi / Keuangan,Audit & Pajak","Tip;Asuransi kesehatan;Waktu regular, Senin - ...",...,Konstruksi/Bangunan/Teknik,ResponsibilitiesPrepare corporate income tax (...,8000000.0,responsibilitiesprepare corporate income tax c...,{'text': 'responsibilitiesprepare corporate in...,"['income tax', 'tax compliance', 'digital elec...",Supervisor,3,Middle,S1
1,27,MARKETING COMMUNICATION & CHANNEL MANAGER,Jakarta Pusat,IDR,Manajer/Asisten Manajer,2 tahun,Tidak terspesifikasi,Penuh Waktu,"Penjualan / Pemasaran,Pemasaran/Pengembangan B...",Asuransi Gigi;Asuransi kesehatan;Parkir;Waktu ...,...,"Konsultasi (IT, Ilmu Pengetahuan, Teknis & Tek...","SCOPE OF ROLE :To plan, develop, implement and...",15000000.0,scope of role to plan develop implement and me...,{'text': 'scope of role to plan develop implem...,"['marketing communication', 'strategic communi...",Manajer,2,Middle,Unknown
2,32,PPIC,Tangerang,IDR,Manajer/Asisten Manajer,5 tahun,Sarjana (S1),Penuh Waktu,"Manufaktur,Manufaktur","Asuransi kesehatan;Waktu regular, Senin - Juma...",...,Manufaktur/Produksi,Tanggung Jawab:Membuat perencanaan produksi be...,6600000.0,tanggung jawab membuat perencanaan produksi be...,{'text': 'tanggung jawab membuat perencanaan p...,"['warehouse management', 'inventory control', ...",Manajer,5,Senior,S1
3,35,C# Advanced/Senior Developer,Jakarta Timur,IDR,Supervisor/Koordinator,2 tahun,"SMA, SMU/SMK/STM, Sertifikat Professional, D3 ...",Penuh Waktu,"Komputer/Teknologi Informasi,IT-Perangkat Lunak","Tip;Asuransi kesehatan;Waktu regular, Senin - ...",...,Komputer/Teknik Informatika (Perangkat Lunak),Kualifikasi:Kandidat harus memiliki setidaknya...,9500000.0,kualifikasi kandidat harus memiliki setidaknya...,{'text': 'kualifikasi kandidat harus memiliki ...,"['asp net', 'object orient programming', 'c', ...",Supervisor,2,Middle,S1
4,38,IT Software Developer Staff,Jakarta Barat,IDR,Pegawai (non-manajemen & non-supervisor),1 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Komputer/Teknologi Informasi,IT-Perangkat Lunak",Kasual (contoh: Kaos);Senin - Jum'at (HO) Seni...,...,Industri Berat/Mesin/Peralatan,Bachelor’s degree of Information Technology or...,6800000.0,bachelor’s degree of information technology or...,{'text': 'bachelor’s degree of information tec...,"['information technology', 'restful api', 'spr...",Staff,1,Entry,S2


In [3]:
# Get a NumPy array of all unique values
unique_values_list = df['job_function'].unique()
print(unique_values_list)

['Akuntansi / Keuangan,Audit & Pajak'
 'Penjualan / Pemasaran,Pemasaran/Pengembangan Bisnis'
 'Manufaktur,Manufaktur' 'Komputer/Teknologi Informasi,IT-Perangkat Lunak'
 'Pelayanan,Logistik/Rantai Pasokan'
 'Penjualan / Pemasaran,Digital Marketing'
 'Penjualan / Pemasaran,Penjualan - Jasa Keuangan'
 'Komputer/Teknologi Informasi,IT-Admin Jaringan/Sistem/Database'
 'Akuntansi / Keuangan,Akuntansi Umum / Pembiayaan'
 'Sains,Aktuaria/Statistik' 'Manufaktur,Pemeliharaan'
 'Teknik,Teknik Lingkungan'
 'Penjualan / Pemasaran,Penjualan - Teknik/Teknikal/IT'
 'Teknik,Teknik Lainnya' 'Seni/Media/Komunikasi,Seni / Desain Kreatif'
 'Penjualan / Pemasaran,Penjualan Ritel'
 'Manufaktur,Pembelian/Manajemen Material'
 'Bangunan/Konstruksi,Teknik Sipil/Konstruksi Bangunan'
 'Sumber Daya Manusia/Personalia,Sumber Daya Manusia / HR'
 'Akuntansi / Keuangan,Perbankan / Jasa Finansial '
 'Penjualan / Pemasaran,Merchandising' 'Teknik,Mekanikal'
 'Penjualan / Pemasaran,Penjualan - Korporasi'
 'Sains,Teknologi 

In [4]:
job_function_mapping = {

    # Finance & Accounting
    "Akuntansi / Keuangan,Audit & Pajak": "Finance & Accounting",
    "Akuntansi / Keuangan,Akuntansi Umum / Pembiayaan": "Finance & Accounting",
    "Akuntansi / Keuangan,Perbankan / Jasa Finansial ": "Finance & Accounting",
    "Akuntansi / Keuangan,Keuangan / Investasi Perusahaan ": "Finance & Accounting",
    "Sains,Aktuaria/Statistik": "Finance & Accounting",

    # IT & Software
    "Komputer/Teknologi Informasi,IT-Perangkat Lunak": "IT & Software",
    "Komputer/Teknologi Informasi,IT-Admin Jaringan/Sistem/Database": "IT & Software",
    "Komputer/Teknologi Informasi,IT-Perangkat Keras": "IT & Software",

    # Engineering (Non-IT)
    "Teknik,Teknik Lingkungan": "Engineering",
    "Teknik,Teknik Lainnya": "Engineering",
    "Teknik,Mekanikal": "Engineering",
    "Teknik,Teknik Elektronika": "Engineering",
    "Teknik,Teknik Elektro": "Engineering",
    "Teknik,Teknik Industri": "Engineering",
    "Teknik,Teknik Kimia": "Engineering",
    "Bangunan/Konstruksi,Teknik Sipil/Konstruksi Bangunan": "Engineering",
    "Bangunan/Konstruksi,Survei Kuantitas": "Engineering",

    # Manufacturing & Operations
    "Manufaktur,Manufaktur": "Manufacturing & Operations",
    "Manufaktur,Pemeliharaan": "Manufacturing & Operations",
    "Manufaktur,Pembelian/Manajemen Material": "Manufacturing & Operations",
    "Manufaktur,Penjaminan Kualitas / QA": "Manufacturing & Operations",
    "Pelayanan,Logistik/Rantai Pasokan": "Manufacturing & Operations",

    # Sales & Marketing
    "Penjualan / Pemasaran,Pemasaran/Pengembangan Bisnis": "Sales & Marketing",
    "Penjualan / Pemasaran,Digital Marketing": "Sales & Marketing",
    "Penjualan / Pemasaran,Penjualan - Jasa Keuangan": "Sales & Marketing",
    "Penjualan / Pemasaran,Penjualan - Teknik/Teknikal/IT": "Sales & Marketing",
    "Penjualan / Pemasaran,Penjualan Ritel": "Sales & Marketing",
    "Penjualan / Pemasaran,Merchandising": "Sales & Marketing",
    "Penjualan / Pemasaran,Penjualan - Korporasi": "Sales & Marketing",
    "Penjualan / Pemasaran,Telesales/Telemarketing": "Sales & Marketing",
    "Penjualan / Pemasaran,E-commerce": "Sales & Marketing",
    "Seni/Media/Komunikasi,Periklanan": "Sales & Marketing",

    # HR & Administration
    "Sumber Daya Manusia/Personalia,Sumber Daya Manusia / HR": "HR & Administration",
    "Sumber Daya Manusia/Personalia,Staf / Administrasi umum": "HR & Administration",
    "Sumber Daya Manusia/Personalia,Sekretaris": "HR & Administration",
    "Sumber Daya Manusia/Personalia,Top Management / Manajemen Tingkat Atas": "HR & Administration",

    # Creative & Media
    "Seni/Media/Komunikasi,Seni / Desain Kreatif": "Creative & Media",
    "Seni/Media/Komunikasi,Hubungan Masyarakat": "Creative & Media",
    "Seni/Media/Komunikasi,Hiburan": "Creative & Media",
    "Lainnya,Jurnalis/Editor": "Creative & Media",

    # Healthcare & Life Sciences
    "Layanan Kesehatan,Diagnosa/Lainnya": "Healthcare",
    "Layanan Kesehatan,Farmasi": "Healthcare",
    "Layanan Kesehatan,Praktisi/Asisten Medis": "Healthcare",
    "Sains,Bioteknologi": "Healthcare",
    "Sains,Biomedis": "Healthcare",
    "Sains,Kimia": "Healthcare",
    "Sains,Geologi/Geofisika": "Healthcare",

    # Education & Training
    "Pendidikan/Pelatihan,Pendidikan": "Education",
    "Pendidikan/Pelatihan,Pelatihan & Pengembangan": "Education",

    # Hospitality & Service
    "Hotel/Restoran,Makanan/Minuman/Pelayanan Restoran": "Hospitality & Service",
    "Hotel/Restoran,Hotel/Pariwisata": "Hospitality & Service",
    "Pelayanan,Layanan Pelanggan": "Hospitality & Service",
    "Pelayanan,Teknikal & Bantuan Pelanggan": "Hospitality & Service",
    "Pelayanan,Perawatan Pribadi": "Hospitality & Service",

    # Legal & Public Service
    "Pelayanan,Pengacara / Asisten Legal": "Legal & Public Service",
    "Pelayanan,Angkatan Bersenjata": "Legal & Public Service",

    # Science & Agriculture
    "Sains,Pertanian": "Science & Agriculture",
    "Sains,Teknologi Makanan/Ahli Gizi": "Science & Agriculture",
    "Sains,Sains & Teknologi": "Science & Agriculture",

    # Other
    "Bangunan/Konstruksi,Arsitek/Desain Interior": "Other",
    "Bangunan/Konstruksi,Properti/Real Estate": "Other",
    "Lainnya,Pekerjaan Umum": "Other",
    "Lainnya,Lainnya/Kategori tidak tersedia": "Other",
}

df["job_function_group"] = (
    df["job_function"]
    .map(job_function_mapping)
    .fillna("Other")
)

In [5]:
encoders_dict = {}

columns_to_encode = ['employment_type', 'career_level_norm', 'experience_years', 'experience_bucket', 'education_level_norm',
                    'job_function', 'job_function_group']
for col in columns_to_encode:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    encoders_dict[col] = le

df.head()

,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,...,job_description,salary,job_description_cleaned,annotations,skills,career_level_norm,experience_years,experience_bucket,education_level_norm,job_function_group
0,10,Tax Supervisor,Banten,IDR,Supervisor/Koordinator,3 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",3,1,"Tip;Asuransi kesehatan;Waktu regular, Senin - ...",...,ResponsibilitiesPrepare corporate income tax (...,8000000.0,responsibilitiesprepare corporate income tax c...,{'text': 'responsibilitiesprepare corporate in...,"['income tax', 'tax compliance', 'digital elec...",4,2,1,1,3
1,27,MARKETING COMMUNICATION & CHANNEL MANAGER,Jakarta Pusat,IDR,Manajer/Asisten Manajer,2 tahun,Tidak terspesifikasi,3,34,Asuransi Gigi;Asuransi kesehatan;Parkir;Waktu ...,...,"SCOPE OF ROLE :To plan, develop, implement and...",15000000.0,scope of role to plan develop implement and me...,{'text': 'scope of role to plan develop implem...,"['marketing communication', 'strategic communi...",2,1,1,5,11
2,32,PPIC,Tangerang,IDR,Manajer/Asisten Manajer,5 tahun,Sarjana (S1),3,19,"Asuransi kesehatan;Waktu regular, Senin - Juma...",...,Tanggung Jawab:Membuat perencanaan produksi be...,6600000.0,tanggung jawab membuat perencanaan produksi be...,{'text': 'tanggung jawab membuat perencanaan p...,"['warehouse management', 'inventory control', ...",2,4,2,1,9
3,35,C# Advanced/Senior Developer,Jakarta Timur,IDR,Supervisor/Koordinator,2 tahun,"SMA, SMU/SMK/STM, Sertifikat Professional, D3 ...",3,12,"Tip;Asuransi kesehatan;Waktu regular, Senin - ...",...,Kualifikasi:Kandidat harus memiliki setidaknya...,9500000.0,kualifikasi kandidat harus memiliki setidaknya...,{'text': 'kualifikasi kandidat harus memiliki ...,"['asp net', 'object orient programming', 'c', ...",4,1,1,1,7
4,38,IT Software Developer Staff,Jakarta Barat,IDR,Pegawai (non-manajemen & non-supervisor),1 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",3,12,Kasual (contoh: Kaos);Senin - Jum'at (HO) Seni...,...,Bachelor’s degree of Information Technology or...,6800000.0,bachelor’s degree of information technology or...,{'text': 'bachelor’s degree of information tec...,"['information technology', 'restful api', 'spr...",3,0,0,2,7


In [6]:
df["role_text"] = df["job_title"] + " - " + df['skills'] + " - " + df["job_description_cleaned"]

In [7]:
EMBED_MODEL_NAME = "paraphrase-multilingual-MiniLM-L12-v2"
embed_model = SentenceTransformer(EMBED_MODEL_NAME)

def embed_texts(texts, batch_size=64):
    return embed_model.encode(texts, show_progress_bar=True, batch_size=batch_size, convert_to_numpy=True)

role_embeddings = embed_texts(df["role_text"].astype(str).tolist())

df["role_embedding"] = list(role_embeddings)

Batches:   0%|          | 0/46 [00:00<?, ?it/s]

In [8]:
df.head()

,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,...,job_description_cleaned,annotations,skills,career_level_norm,experience_years,experience_bucket,education_level_norm,job_function_group,role_text,role_embedding
0,10,Tax Supervisor,Banten,IDR,Supervisor/Koordinator,3 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",3,1,"Tip;Asuransi kesehatan;Waktu regular, Senin - ...",...,responsibilitiesprepare corporate income tax c...,{'text': 'responsibilitiesprepare corporate in...,"['income tax', 'tax compliance', 'digital elec...",4,2,1,1,3,"Tax Supervisor - ['income tax', 'tax complianc...","[0.010884536, 0.23534577, -0.064613506, 0.0986..."
1,27,MARKETING COMMUNICATION & CHANNEL MANAGER,Jakarta Pusat,IDR,Manajer/Asisten Manajer,2 tahun,Tidak terspesifikasi,3,34,Asuransi Gigi;Asuransi kesehatan;Parkir;Waktu ...,...,scope of role to plan develop implement and me...,{'text': 'scope of role to plan develop implem...,"['marketing communication', 'strategic communi...",2,1,1,5,11,MARKETING COMMUNICATION & CHANNEL MANAGER - ['...,"[0.18302445, 0.13662589, 0.047362998, -0.16890..."
2,32,PPIC,Tangerang,IDR,Manajer/Asisten Manajer,5 tahun,Sarjana (S1),3,19,"Asuransi kesehatan;Waktu regular, Senin - Juma...",...,tanggung jawab membuat perencanaan produksi be...,{'text': 'tanggung jawab membuat perencanaan p...,"['warehouse management', 'inventory control', ...",2,4,2,1,9,"PPIC - ['warehouse management', 'inventory con...","[-0.31862533, -0.08181146, -0.08548966, -0.078..."
3,35,C# Advanced/Senior Developer,Jakarta Timur,IDR,Supervisor/Koordinator,2 tahun,"SMA, SMU/SMK/STM, Sertifikat Professional, D3 ...",3,12,"Tip;Asuransi kesehatan;Waktu regular, Senin - ...",...,kualifikasi kandidat harus memiliki setidaknya...,{'text': 'kualifikasi kandidat harus memiliki ...,"['asp net', 'object orient programming', 'c', ...",4,1,1,1,7,"C# Advanced/Senior Developer - ['asp net', 'ob...","[0.11805347, 0.20948735, -0.028469969, -0.0280..."
4,38,IT Software Developer Staff,Jakarta Barat,IDR,Pegawai (non-manajemen & non-supervisor),1 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",3,12,Kasual (contoh: Kaos);Senin - Jum'at (HO) Seni...,...,bachelor’s degree of information technology or...,{'text': 'bachelor’s degree of information tec...,"['information technology', 'restful api', 'spr...",3,0,0,2,7,IT Software Developer Staff - ['information te...,"[0.016292084, -0.035321087, -0.027727457, -0.1..."


In [9]:
X_emb = np.hstack([role_embeddings])
print("X_emb shape:", X_emb.shape)

num_features = ['employment_type', 'career_level_norm', 'experience_years', 'experience_bucket', 'education_level_norm']

if len(num_features) > 0:
    cat_values = df[num_features].values.astype(np.float32)
    X = np.hstack([X_emb, cat_values])
else:
    X = X_emb

print("Final feature X shape:", X.shape)

y = df["job_function_group"].fillna("unknown")
print("n_classes:", y.nunique())
print(y.value_counts().head())

X_emb shape: (2921, 384)
Final feature X shape: (2921, 389)
n_classes: 13
job_function_group
11    1038
7      598
3      283
9      243
4      196
Name: count, dtype: int64


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_emb, y, test_size=0.3, random_state=42)

In [11]:
# Train the Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Evaluate the model
accuracy_before = rf.score(X_test, y_test)
print(f'Accuracy before feature selection: {accuracy_before:.2f}')

Accuracy before feature selection: 0.83


In [12]:
embedding_dim = X_emb.shape[1]
feature_names = (
    [f"role_embedding_{i}" for i in range(embedding_dim)]
    # + num_features
)

feature_importance_df = pd.DataFrame({
    "Feature": feature_names,
    "Importance": rf.feature_importances_
}).sort_values("Importance", ascending=False)
print(feature_importance_df)

# Select top N features (example selecting top 10 features)
top_feature_indices = feature_importance_df.index[:10].values
X_train_selected = X_train[:, top_feature_indices]
X_test_selected = X_test[:, top_feature_indices]

                Feature  Importance
47    role_embedding_47    0.018891
109  role_embedding_109    0.017346
56    role_embedding_56    0.014010
271  role_embedding_271    0.013434
25    role_embedding_25    0.012313
..                  ...         ...
190  role_embedding_190    0.001005
351  role_embedding_351    0.000987
208  role_embedding_208    0.000975
196  role_embedding_196    0.000950
114  role_embedding_114    0.000896

[384 rows x 2 columns]


In [13]:
# Train the Random Forest model with selected features
rf_selected = RandomForestClassifier(n_estimators=100, random_state=42)
rf_selected.fit(X_train_selected, y_train)

# Evaluate the model
accuracy_after = rf_selected.score(X_test_selected, y_test)
print(f'Accuracy after feature selection: {accuracy_after:.2f}')

Accuracy after feature selection: 0.75


## TFIDF

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
tfidf = TfidfVectorizer(
    max_features=50,        # penting agar RF tidak overload
    ngram_range=(1, 2),
    # stop_words="english",
    min_df=2,
    max_df=0.9,
    sublinear_tf=True
)

In [16]:
X_tfidf = tfidf.fit_transform(df["role_text"].astype(str))

In [17]:
X_tfidf_dense = X_tfidf.toarray()
X_emb = X_tfidf_dense
print("TF-IDF shape:", X_emb.shape)

TF-IDF shape: (2921, 50)


In [18]:
num_features = ['employment_type', 'career_level_norm', 'experience_years', 'experience_bucket', 'education_level_norm',
                    'job_function']

if len(num_features) > 0:
    cat_values = df[num_features].values.astype(np.float32)
    X = np.hstack([X_emb, cat_values])
else:
    X = X_emb

y = df["company_industry_simple"].fillna("unknown")
print("Final X shape:", X.shape)

KeyError: 'company_industry_simple'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Train the Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Evaluate the model
accuracy_before = rf.score(X_test, y_test)
print(f'Accuracy before feature selection: {accuracy_before:.2f}')

In [ ]:
tfidf_feature_names = tfidf.get_feature_names_out()

feature_names = list(tfidf_feature_names) + num_features
feature_importance_df = pd.DataFrame({
    "Feature": feature_names,
    "Importance": rf.feature_importances_
}).sort_values("Importance", ascending=False)

In [ ]:
print(feature_importance_df)

top_feature_indices = feature_importance_df.index[:10].values

X_train_selected = X_train[:, top_feature_indices]
X_test_selected = X_test[:, top_feature_indices]

In [ ]:
# Train the Random Forest model with selected features
rf_selected = RandomForestClassifier(n_estimators=100, random_state=42)
rf_selected.fit(X_train_selected, y_train)

# Evaluate the model
accuracy_after = rf_selected.score(X_test_selected, y_test)
print(f'Accuracy after feature selection: {accuracy_after:.2f}')

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(
    max_iter=3000,
    n_jobs=-1,
    class_weight="balanced",
    solver="lbfgs"
)

clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
tfidf_features = tfidf.get_feature_names_out()
all_features = list(tfidf_features) + num_features

In [ ]:
import numpy as np

class_idx = list(clf.classes_).index(5)  # ganti sesuai kelas

coef = clf.coef_[class_idx]
top_idx = np.argsort(coef)[-20:]

for i in reversed(top_idx):
    print(all_features[i], coef[i])

In [ ]:
df.info()

In [ ]:
import lightgbm as lgb

model = lgb.LGBMClassifier(
    objective="multiclass",
    num_class=y.nunique(),
    n_estimators=400,
    learning_rate=0.05,
    num_leaves=63,
    max_depth=-1,
    subsample=0.8,
    colsample_bytree=0.8,
    n_jobs=-1,
    class_weight="balanced"
)

model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

#### Deep Learning

In [ ]:
text = (
    df["job_title"].fillna("") + " " +
    df["skills"].fillna("") + " " +
    df["job_description_cleaned"].fillna("")
).tolist()

In [ ]:
y = df["company_industry_simple"]

In [ ]:
from sentence_transformers import SentenceTransformer

EMBED_MODEL_NAME = "paraphrase-multilingual-MiniLM-L12-v2"
embed_model = SentenceTransformer(EMBED_MODEL_NAME)

In [ ]:
X_emb = embed_model.encode(
    text,
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

print("Embedding shape:", X_emb.shape)
# (n_samples, 384)

In [ ]:
import numpy as np

meta_cols = ['employment_type',
             'career_level_norm',
             'experience_years',
             'experience_bucket',
             'education_level_norm',
             'job_function']

X_meta = df[meta_cols].values.astype(np.float32)

X = np.hstack([X_emb, X_meta])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(
    max_iter=3000,
    n_jobs=-1,
    class_weight="balanced",
    multi_class="auto"
)

clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
import torch
import numpy as np

X = torch.tensor(X_emb, dtype=torch.float32)
y = torch.tensor(pd.factorize(y)[0], dtype=torch.long)

num_classes = len(torch.unique(y))

In [ ]:
import torch.nn as nn

class SBERTClassifier(nn.Module):
    def __init__(self, input_dim=384, num_classes=10):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.4),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
model = SBERTClassifier(
    input_dim=X.shape[1],
    num_classes=num_classes
)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

for epoch in range(20):
    optimizer.zero_grad()
    logits = model(X)
    loss = criterion(logits, y)
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1} | Loss: {loss.item():.4f}")

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

model.eval()
with torch.no_grad():
    logits = model(X_test)
    preds = torch.argmax(logits, dim=1)

acc = accuracy_score(y_test, preds)
f1 = f1_score(y_test, preds, average="macro")

print("Accuracy:", acc)
print("Macro F1:", f1)

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

X_vis = TSNE(n_components=2, perplexity=30, random_state=42).fit_transform(X_emb)

plt.figure(figsize=(10, 8))
plt.scatter(X_vis[:,0], X_vis[:,1], c=y, cmap="tab20", s=5)
plt.title("SBERT Embedding Visualization")
plt.show()

In [ ]:
df.info()

In [ ]:
from sklearn.preprocessing import LabelEncoder

group_encoder = LabelEncoder()
sub_encoder = LabelEncoder()

df["group_id"] = group_encoder.fit_transform(df["company_industry_simple"])
df["sub_id"] = sub_encoder.fit_transform(df["company_industry"])

In [ ]:
X_emb = embed_model.encode(
    df["role_text"].tolist(),
    normalize_embeddings=True,
    convert_to_numpy=True
)

In [ ]:
from sklearn.linear_model import LogisticRegression

clf_group = LogisticRegression(
    max_iter=2000,
    class_weight="balanced"
)

clf_group.fit(X_emb, df["group_id"])

In [ ]:
sub_models = {}
single_sub_mapping = {}

for group in df["company_industry_simple"].unique():
    idx = df["company_industry_simple"] == group

    X_g = X_emb[idx]
    y_g = df.loc[idx, "sub_id"]

    if y_g.nunique() == 1:
        # hanya satu sub-industry → tidak perlu model
        single_sub_mapping[group] = y_g.iloc[0]
        print(f"[SKIP] {group}: only one sub-industry")
        continue

    clf = LogisticRegression(
        max_iter=2000,
        class_weight="balanced"
    )
    clf.fit(X_g, y_g)

    sub_models[group] = clf

In [ ]:
def predict_industry(text):
    emb = embed_model.encode(
        [text],
        normalize_embeddings=True
    )

    # Level-1
    group_id = clf_group.predict(emb)[0]
    group_name = group_encoder.inverse_transform([group_id])[0]

    # Level-2
    if group_name in single_sub_mapping:
        sub_id = single_sub_mapping[group_name]
    else:
        clf_sub = sub_models[group_name]
        sub_id = clf_sub.predict(emb)[0]

    sub_name = sub_encoder.inverse_transform([sub_id])[0]

    return group_name, sub_name

In [ ]:
df.groupby("company_industry_simple")["company_industry"].nunique().sort_values()

In [ ]:
from sklearn.metrics import accuracy_score

group_pred = clf_group.predict(X_emb)
print("Group Accuracy:", accuracy_score(df["group_id"], group_pred))

In [ ]:
correct = 0
total = 0

for group, clf in sub_models.items():
    idx = df["company_industry_simple"] == group
    X_g = X_emb[idx]
    y_g = df.loc[idx, "sub_id"]

    preds = clf.predict(X_g)
    correct += (preds == y_g).sum()
    total += len(y_g)

print("Conditional Sub Accuracy:", correct / total)

In [ ]:
overall = (group_pred == df["group_id"]).mean()
print("Overall Hierarchical Accuracy:", overall)

In [ ]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=20, metric="cosine")
nn.fit(X_emb)

def predict_industry_knn(text):
    emb = embed_model.encode([text], normalize_embeddings=True)
    dist, idx = nn.kneighbors(emb)

    labels = df.iloc[idx[0]]["company_industry_simple"]
    return labels.value_counts().idxmax()

In [ ]:
X_text = df["role_text"]          # atau job_description_cleaned
y = df["company_industry_simple"] # target

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

X_emb = model.encode(
    X_text.tolist(),
    batch_size=32,
    show_progress_bar=True,
    normalize_embeddings=True  # PENTING untuk cosine
)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_emb, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(
    n_neighbors=15,
    metric="cosine",
    weights="distance"  # ini KRUSIAL
)

knn.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

y_pred = knn.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
import numpy as np

for k in [5, 10, 15, 20, 30]:
    knn = KNeighborsClassifier(
        n_neighbors=k,
        metric="cosine",
        weights="distance"
    )
    knn.fit(X_train, y_train)
    acc = knn.score(X_test, y_test)
    print(f"k={k}, acc={acc:.3f}")

In [ ]:
import numpy as np

def top_k_accuracy(model, X, y, k=3):
    probs = model.predict_proba(X)
    topk = np.argsort(probs, axis=1)[:, -k:]
    return np.mean([
        y.iloc[i] in model.classes_[topk[i]]
        for i in range(len(y))
    ])

print("Top-3 Accuracy:", top_k_accuracy(knn, X_test, y_test, k=3))

In [ ]:
idx = 45
dist, neighbors = knn.kneighbors([X_test[idx]], n_neighbors=5)

df.iloc[neighbors[0]][["role_text", "company_industry_simple"]]